In [2]:
import json, requests

# Caminho relativo para o arquivo JSON
path = 'symbol_exchanges_dict.json'


with open(path, 'r') as json_file:
    symbol_exchanges_dict = json.load(json_file)


symbol_exchanges_dict
sumary_results = []

for exchange_name in symbol_exchanges_dict['btcbrl']:
    base_url = f"https://api.cryptowat.ch/markets/{exchange_name}/btcbrl/summary"
    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()
        data['exchange_name'] = exchange_name
        sumary_results.append(data)
    else:
        sumary_results.append(None)

print(sumary_results)


[{'result': {'price': {'last': 141682.7351, 'high': 152102.7456, 'low': 141682.7351, 'change': {'percentage': -0.06719146417766352, 'absolute': -10205.599600000016}}, 'volume': 0.265100000000003, 'volumeQuote': 39246.55590935982}, 'allowance': {'cost': 0.005, 'remaining': 9.995, 'upgrade': 'For unlimited API access, create an account at https://cryptowat.ch'}, 'exchange_name': 'bisq'}, {'result': {'price': {'last': 146663, 'high': 146989, 'low': 146287, 'change': {'percentage': -0.0003680555082233143, 'absolute': -54}}, 'volume': 40.743329999908006, 'volumeQuote': 5975179.0876807785}, 'allowance': {'cost': 0.005, 'remaining': 9.99, 'upgrade': 'For unlimited API access, create an account at https://cryptowat.ch'}, 'exchange_name': 'binance'}, {'result': {'price': {'last': 146613, 'high': 147139, 'low': 146268, 'change': {'percentage': -0.001226216508961599, 'absolute': -180}}, 'volume': 5.211156999999866, 'volumeQuote': 764278.8033840092}, 'allowance': {'cost': 0.005, 'remaining': 9.985

In [16]:
!pip install findspark boto3 minio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.9 MB/s eta 0:00:00


In [14]:
%%time

import boto3
import findspark
import os
import pandas as pd

findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.pandas as ps

from IPython.core.display import HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))

os.environ['PYARROW_IGNORE_TIMEZONE'] = str(1)

spark = SparkSession.builder \
    .appName("testing") \
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.access.key", 'airflow') \
    .config("spark.hadoop.fs.s3a.secret.key", 'sample_key') \
    .config("spark.hadoop.fs.s3a.proxy.host", "minio1") \
    .config("spark.hadoop.fs.s3a.endpoint", "minio1") \
    .config("spark.hadoop.fs.s3a.proxy.port", "9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

print("Spark inicializado com sucesso com as credenciais do S3")

Spark inicializado com sucesso com as credenciais do S3
CPU times: user 68.7 ms, sys: 20.2 ms, total: 88.9 ms
Wall time: 120 ms


In [8]:
exch = spark.read.json("s3a://raw/symbol_exchanges_dict.json")

In [39]:
import json
from minio import Minio
import datetime

client = Minio('minio1:9000',
               access_key='airflow',
               secret_key='sample_key',
               secure=False)

object_data = client.get_object('raw', 'symbol_exchanges_dict.json')
json_data = object_data.data.decode('utf-8')
symbol_exchanges_dict=eval(json_data)

import json, requests, io
for exchange_name in symbol_exchanges_dict['btcbrl']:
    base_url = f"https://api.cryptowat.ch/markets/{exchange_name}/btcbrl/summary"
    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()
        data['exchange_name'] = exchange_name
        # current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        file_obj = io.BytesIO(str(data).encode())
        client.put_object('raw',f'exchanges/{exchange_name}.json',file_obj, file_obj.getbuffer().nbytes)
    else:
        print("Não foi possível obter o exchange")

2023-07-16_02-50-16


In [96]:
import findspark
import os
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *
from minio import Minio

os.environ['PYARROW_IGNORE_TIMEZONE'] = str(1)

spark = SparkSession.builder \
    .appName("elt_raw_silver_exchanges") \
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.access.key", 'airflow') \
    .config("spark.hadoop.fs.s3a.secret.key", 'sample_key') \
    .config("spark.hadoop.fs.s3a.proxy.host", "minio1") \
    .config("spark.hadoop.fs.s3a.endpoint", "minio1") \
    .config("spark.hadoop.fs.s3a.proxy.port", "9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

client = Minio('minio1:9000',
               access_key='airflow',
               secret_key='sample_key',
               secure=False)

def flatten_test(df, field, sep="_"):

    # compute Complex Fields (Arrays, Structs and Maptypes) in Schema
    complex_fields = dict(
        [
            (field.name, field.dataType)
            for field in df.schema.fields
            if type(field.dataType) == ArrayType
            or type(field.dataType) == StructType
            or type(field.dataType) == MapType
        ]
    )

    while len(complex_fields) != 0:
        col_name = list(complex_fields.keys())[0]
        # print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))

        # if StructType then convert all sub element to columns.
        # i.e. flatten structs
        if type(complex_fields[col_name]) == StructType:
            expanded = [
                col(col_name + "." + k).alias(col_name + sep + k)
                for k in [n.name for n in complex_fields[col_name]]
            ]
            df = df.select("*", *expanded).drop(col_name)

        # if ArrayType then add the Array Elements as Rows using the explode function
        # i.e. explode Arrays
        elif type(complex_fields[col_name]) == ArrayType:
            df = df.withColumn(col_name, explode_outer(col_name))

        # if MapType then convert all sub element to columns.
        # i.e. flatten
        elif type(complex_fields[col_name]) == MapType:
            keys_df = df.select(explode_outer(map_keys(col(col_name)))).distinct()
            keys = list(map(lambda row: row[0], keys_df.collect()))
            key_cols = list(
                map(
                    lambda f: col(col_name).getItem(f).alias(str(col_name + sep + f)),
                    keys,
                )
            )
            drop_column_list = [col_name]
            df = df.select(
                [
                    col_name
                    for col_name in df.columns
                    if col_name not in drop_column_list
                ]
                + key_cols
            )

        # recompute remaining Complex Fields in Schema
        complex_fields = dict(
            [
                (field.name, field.dataType)
                for field in df.schema.fields
                if type(field.dataType) == ArrayType
                or type(field.dataType) == StructType
                or type(field.dataType) == MapType
            ]
        )

    return df


for obj in client.list_objects('raw', prefix=f'exchanges/'):
    exch = spark.read.json(f"s3a://raw/{obj.object_name}")
    exch = exch.drop('allowance')
    fields = exch.select("result").schema.fields
    df_flat = flatten_test(exch,fields)
    df_flat = df_flat.withColumn('datetime', lit(obj.last_modified))
    df_flat.write.parquet(f"s3a://silver/{obj.object_name[:-5]}.parquet")

exchanges/binance


In [94]:

df_flat.printSchema()

root
 |-- exchange_name: string (nullable = true)
 |-- result_volume: double (nullable = true)
 |-- result_volumeQuote: double (nullable = true)
 |-- result_price_high: long (nullable = true)
 |-- result_price_last: long (nullable = true)
 |-- result_price_low: long (nullable = true)
 |-- result_price_change_absolute: long (nullable = true)
 |-- result_price_change_percentage: double (nullable = true)



In [95]:
df_flat.show()

+-------------+-----------------+------------------+-----------------+-----------------+----------------+----------------------------+------------------------------+
|exchange_name|    result_volume|result_volumeQuote|result_price_high|result_price_last|result_price_low|result_price_change_absolute|result_price_change_percentage|
+-------------+-----------------+------------------+-----------------+-----------------+----------------+----------------------------+------------------------------+
|      binance|45.80987999990816| 6714543.838720769|           146989|           146329|          145645|                         -14|          -9.56656621772138E-5|
+-------------+-----------------+------------------+-----------------+-----------------+----------------+----------------------------+------------------------------+

